## Inicializacao

In [2]:
import numpy as np
import pandas as pd
import sympy as sp
import scipy.io as sc 
from sympy import symbols, pprint
import matplotlib.pyplot as plt
from structureSelector import *
from methods.utils.utilities import *

def metrics(y, yest):
    residuo1 = y - yest
    mape = round(np.mean(np.abs(residuo1 / (yest + np.finfo(np.float64).eps))) * 100, 3)
    print('MSE:', np.mean(np.square(residuo1)), '\nAET:', np.sum(np.abs(residuo1)), '\nMAPE:', str(mape) + '%')
    cc = np.corrcoef(y, yest)
    #print("Correlation pearson:", np.mean(cc))

dataTank = pd.read_csv('data/coupletanks.csv')
u = np.reshape(np.array(dataTank['u']), (1,2401))
y = np.array(dataTank[['tank1', 'tank2']].T)

## Selecao de estrutura

In [ ]:
#Selecione o tanque 
output = 0  # 0 ou 1

num = [3, 5]
params = []
params.append({'nb':[2,2],'na':[12], 'level':1, 'nonlinear':[0,0,0,0,0], 'root':True, 'delay':8, 'diff':True})
params.append({'nb':[0,2],'na':[1], 'level':1, 'nonlinear':[0,0,0,0,0], 'root':True, 'delay':0, 'diff':True})

sselector = structureSelector()
ss = sselector.symbolic_regressors(**params[output])

vCandidatos = sselector.matrix_candidate(u, y, **params[output], dt=0.1)

pad = max(max(params[output]['nb']), max(params[output]['na']))
psi, selected  = sselector.semp(vCandidatos.T, y[output, pad:], num[output], 0.00001)
theta = LSM(y[output, pad:], psi)
model = ss[selected]
print(model, theta)

### Simulação

In [ ]:

slivre = sselector.predict(u, y, theta, ss[selected], params[output]['nb'], params[output]['na'], output, params[output]['delay'], params[output]['diff'], dt=0.1)
yhat = sselector.oneStepForward(u, y, theta, ss[selected], params[output]['nb'], params[output]['na'], output, diff=True, dt=0.1)

'''index = np.where(y[output,:] != 0)
rs = y[output, index] - slivre[index]
rh = y[output, index] - yhat[index]

print('MAPE LIVRE:', str(round((np.mean(np.abs(rs)/y[output, index])) * 100, 4)) + '%')
print('MAPE OSF:', str(round((np.mean(np.abs(rh)/y[output, index])) * 100, 4)) + '%')

print('MSE LIVRE:', str(round((np.mean(np.abs(rs))), 4)))
print('MSE OSF:', str(round((np.mean(np.abs(rh))), 4)))'''

print("\nSimulação livre")
metrics(y[output], slivre)
print("\nUm passo a frente")
metrics(y[output], yhat)

plt.figure(figsize=(15,4))
plt.title("Tanque " + str(output+1))
plt.plot(y[output].T, label='Original')
plt.plot(yhat, label='um passo a frente')
plt.plot(slivre, label='Livre')
plt.legend()
plt.show()

In [ ]:
r1 = y[output].T - yhat
r2 = y[output].T - slivre
print(np.mean(np.abs(r1)), np.mean(np.abs(r2)))
plt.title('Residuo')
plt.plot(r1, label='um passo a frente')
plt.plot(r2, label='Livre')
plt.legend()
plt.show()

## Validação

In [ ]:
mat_content1 = sc.loadmat("data/ct1x1.mat")
mat_content2 = sc.loadmat("data/ct1x2.mat")

tanque1 = mat_content1['Tanque1']
tanque2 = mat_content2['Tanque2']

t1 = tanque1['time'][0][0]
v1 = tanque1['signals'][0][0]['values'][0][0]

t2 = tanque2['time'][0][0]
v2 = tanque2['signals'][0][0]['values'][0][0]

input = pd.read_csv('data/xinput.csv')
t = input['t']
uVal = np.array(input['v']).reshape((1,-1))

plt.plot(t, uVal.T, label='Entrada')
plt.plot(t1, v1, label="tank 1")
plt.plot(t2, v2, label="tank 2")
plt.legend()
plt.show()

In [ ]:
v1[v1 < 0] = 0
v2[v2 < 0] = 0
v2[:100] = 0
yVal = np.vstack((v1.T, v2.T))
print(yVal.shape)
z = np.zeros(yVal.shape)
valLivre = sselector.predict(uVal, yVal, theta, ss[selected], params[output]['nb'], params[output]['na'], output)
yhat = sselector.oneStepForward(uVal, yVal, theta, ss[selected], params[output]['nb'], params[output]['na'], output)

print("Modelo selecionado:")
pprint( model @ theta)

f, ax = plt.subplots(1,2, figsize=[16,5])

ax[0].plot(yVal[output].T, label='Original')
ax[0].plot(valLivre, label='Livre')
ax[0].plot(yhat, label='um passo a frente')
ax[0].set_title("Simulações")
ax[0].legend()


ax[1].plot(yVal[output].T - valLivre, label='Livre')
ax[1].plot(yVal[output].T - yhat, label='um passo a frente')
ax[1].set_title("Resíduos")
ax[1].legend()
plt.show()

In [ ]:
print("\nSimulação livre")
metrics(yVal[output], valLivre)
print("\nUm passo a frente")
metrics(yVal[output], yhat)

In [ ]:
t = t1[:100]

dt = t[1]-t[0]
h = np.cos(t)
dh = -np.sin(t)
ddh = -np.cos(t)

dy = np.zeros(dh.shape)
ddy = np.zeros(dh.shape)

dy[1:] = (h[1:]-h[:-1])/dt
ddy[2:] = (h[2:] - 2 * h[1:-1] +  h[:-2])/(dt**2) # y[t] - 2*y[t-1] + y[t-2]
#print(ddy)

plt.plot(t, h, label='y')
plt.plot(t, dh, label='derivada')
plt.plot(t, ddh, label='2º derivada')
plt.plot(t, dy, label='aproximação derivada')
plt.plot(t, ddy, label='aproximação da 2º derivada')
plt.legend()
plt.show()

In [ ]:
uu = np.zeros(u.shape)
ry = np.zeros(u.shape)
for i in range(1, u.shape[1]):
    uu[:, i] = uu[:, i-1] + u[:, i] * 0.02 * 0.1
    ry[:, i] = np.sqrt(- 0.002085 * uu[:, i]**2 + 0.3206 * uu[:, i] + 0.251306)


In [ ]:
plt.plot(uu.T)
plt.plot(ry.T)
plt.plot(uu.T - ry.T)
plt.show